### Import libraries

In [2]:
import sys
!{sys.executable} -m pip install pydicom

     |████████████████████████████████| 1.9 MB 18.9 MB/s eta 0:00:01


In [ ]:
!{sys.executable} -m pip install python-dotenv

In [3]:
!{sys.executable} -m pip install opencv-python

In [1]:
import pandas as pd
import os
import shutil
from glob import glob
import pydicom as dicom
import cv2
import uuid
from PIL import Image
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, ContentSettings, __version__

In [2]:
cohen_dir = 'covid-chestxray-dataset'
cohen_csv_path = f'./{cohen_dir}/metadata.csv'
cohen_img_path = f'./{cohen_dir}/images'

In [3]:
fig1_dir = 'Figure1-COVID-chestxray-dataset'
fig1_csv_path = f'./{fig1_dir}/metadata.csv'
fig1_img_path = f'./{fig1_dir}/images'

In [4]:
actmed_dir = 'Actualmed-COVID-chestxray-dataset'
actmed_csv_path = f'./{actmed_dir}/metadata.csv'
actmed_img_path = f'./{actmed_dir}/images'

In [ ]:
sirm_dir = 'COVID-19-Radiography-Database'
sirm_cov_csv_path = f'./{sirm_dir}/COVID.metadata.xlsx'
sirm_cov_img_path = f'./{sirm_dir}/COVID'

In [ ]:
rsna_csv1_path = './stage_2_detailed_class_info.csv'
rsna_csv2_path = './stage_2_train_labels.csv'
rsna_img_path = './stage_2_train_images'

### Initialize directories, files and metrics variables

In [10]:
dataset_dir = 'dataset'

images_dir = f'{dataset_dir}/images'
data_file_name = 'data_file.csv'

deletions = 0

if not os.path.exists(dataset_dir):
    os.mkdir(dataset_dir)
    print('Successfully created \'dataset\' directory created.')

if not os.path.exists(images_dir):
    os.mkdir(images_dir)
    print('Successfully created \'images\' directory created.')
else:
    files = glob(f'{images_dir}/*')
    for f in files:
        try:
            os.remove(f)
            deletions += 1
        except OSError as e:
            print('Error: %s : %s' % (f, e.strerror))
    print(f'Successfully deleted {deletions} images from the \'images\' directory')

data_file = open(f'./{dataset_dir}/{data_file_name}', 'w')
data_file.write('filename,diagnosis\n')
data_file.close()
print(f'Successfully deleted {deletions} entries from \'{data_file_name}\'')

covid_positives = 0
covid_negatives = 0
xray_index = 0

Successfully deleted 463 images from the 'images' directory
Successfully deleted 463 entries from 'data_file.csv'


### Extract data from 'covid-chestray-dataset'

In [6]:
cohen_csv = pd.read_csv(cohen_csv_path, nrows=None)
print(cohen_csv.shape)

data_file = open(f'./{dataset_dir}/{data_file_name}', 'a')

for _, row in cohen_csv.iterrows():
    if row['folder'] == 'volumes' or row['view'] != 'PA':
        continue
    
    image = row['filename']
    
    src_path = f'{cohen_img_path}/{image}'
    dst_path = f'./{images_dir}/{image}'
    
    image_type = image.split('.')[-1]
    new_image_name = f'x-ray-{xray_index}.{image_type}'
    
    shutil.copy(src_path, dst_path)
    os.rename(dst_path, f'./{images_dir}/{new_image_name}')
    
    finding = row['finding'].split('/')[-1]
    
    if finding == 'COVID-19':
        if row['RT_PCR_positive'] != 'Y':
            continue
        sample = new_image_name + ',' + 'positive' + '\n'
        covid_positives += 1
    else:
        sample = new_image_name + ',' + 'negative' + '\n'
        covid_negatives += 1
        
    data_file.write(sample)
    xray_index += 1
    
    print(new_image_name, ': OK')

print('Covid positives:', covid_positives)
print('Covid negatives:', covid_negatives)

data_file.close()

(950, 30)
x-ray-0.jpeg : OK
x-ray-1.jpeg : OK
x-ray-2.jpeg : OK
x-ray-3.jpeg : OK
x-ray-4.jpeg : OK
x-ray-5.jpeg : OK
x-ray-6.png : OK
x-ray-7.jpg : OK
x-ray-8.jpg : OK
x-ray-9.jpeg : OK
x-ray-10.jpeg : OK
x-ray-11.jpeg : OK
x-ray-12.jpg : OK
x-ray-13.jpeg : OK
x-ray-14.jpeg : OK
x-ray-15.jpeg : OK
x-ray-16.jpeg : OK
x-ray-17.jpeg : OK
x-ray-18.jpeg : OK
x-ray-19.jpeg : OK
x-ray-20.jpeg : OK
x-ray-21.jpeg : OK
x-ray-22.jpg : OK
x-ray-23.png : OK
x-ray-24.jpg : OK
x-ray-25.jpeg : OK
x-ray-26.jpeg : OK
x-ray-27.jpeg : OK
x-ray-28.jpeg : OK
x-ray-29.jpeg : OK
x-ray-30.jpeg : OK
x-ray-31.jpeg : OK
x-ray-32.jpeg : OK
x-ray-33.jpeg : OK
x-ray-34.jpeg : OK
x-ray-35.jpeg : OK
x-ray-36.jpeg : OK
x-ray-37.jpeg : OK
x-ray-38.jpeg : OK
x-ray-39.jpg : OK
x-ray-40.jpg : OK
x-ray-41.jpg : OK
x-ray-42.jpg : OK
x-ray-43.png : OK
x-ray-44.png : OK
x-ray-45.png : OK
x-ray-46.png : OK
x-ray-47.png : OK
x-ray-48.png : OK
x-ray-49.jpg : OK
x-ray-50.jpg : OK
x-ray-51.jpeg : OK
x-ray-52.png : OK
x-ray-53.png 

### Extract data from 'Figure1-COVID-chestxray-dataset'

In [7]:
fig1_csv = pd.read_csv(fig1_csv_path, encoding='ISO-8859-1', nrows=None)
print(fig1_csv.shape)

fig1_csv['finding'].fillna('NaN', inplace=True)

data_file = open(f'./{dataset_dir}/{data_file_name}', 'a')

for _, row in fig1_csv.iterrows():
    if row['finding'] == 'NaN':
        continue
    
    patient_id = row['patientid']
    image = f'{patient_id}.jpg'
    
    src_path = f'{fig1_img_path}/{image}'
    if not os.path.exists(src_path):
        image = f'{patient_id}.png'
        src_path = f'{fig1_img_path}/{image}'
    dst_path = f'{images_dir}/{image}'
    
    image_type = image.split('.')[-1]
    new_image_name = f'x-ray-{xray_index}.{image_type}'
    
    shutil.copy(src_path, dst_path)
    os.rename(dst_path, f'./{images_dir}/{new_image_name}')
    
    finding = row['finding']
    
    if finding == 'COVID-19':
        sample = new_image_name + ',' + 'positive' + '\n'
        covid_positives += 1
    else:
        sample = new_image_name + ',' + 'negative' + '\n'
        covid_negatives += 1
    
    data_file.write(sample)
    xray_index += 1
    
    print(new_image_name, ': OK')

print('Covid positives:', covid_positives)
print('Covid negatives:', covid_negatives)

data_file.close()

(55, 12)
x-ray-258.jpg : OK
x-ray-259.jpg : OK
x-ray-260.jpg : OK
x-ray-261.jpg : OK
x-ray-262.jpg : OK
x-ray-263.jpg : OK
x-ray-264.jpg : OK
x-ray-265.png : OK
x-ray-266.png : OK
x-ray-267.jpg : OK
x-ray-268.jpg : OK
x-ray-269.jpg : OK
x-ray-270.jpg : OK
x-ray-271.jpg : OK
x-ray-272.jpg : OK
x-ray-273.jpg : OK
x-ray-274.jpg : OK
x-ray-275.jpg : OK
x-ray-276.jpg : OK
x-ray-277.jpg : OK
x-ray-278.jpg : OK
x-ray-279.jpg : OK
x-ray-280.jpg : OK
x-ray-281.jpg : OK
x-ray-282.jpg : OK
x-ray-283.jpg : OK
x-ray-284.jpg : OK
x-ray-285.jpg : OK
x-ray-286.jpg : OK
x-ray-287.jpg : OK
x-ray-288.jpg : OK
x-ray-289.jpg : OK
x-ray-290.jpg : OK
x-ray-291.jpg : OK
x-ray-292.jpg : OK
x-ray-293.jpg : OK
x-ray-294.jpg : OK
x-ray-295.png : OK
x-ray-296.png : OK
x-ray-297.png : OK
Covid positives: 145
Covid negatives: 153


### Extract data from 'Actualmed-COVID-chestxray-dataset'

In [8]:
actmed_csv = pd.read_csv(actmed_csv_path, nrows=None)
print(actmed_csv.shape)

actmed_csv['finding'].fillna('NaN', inplace=True)

data_file = open(f'./{dataset_dir}/{data_file_name}', 'a')

for _, row in actmed_csv.iterrows():
    if row['finding'] == 'NaN' or row['view'] != 'PA':
        continue
    
    image = row['imagename']
    
    src_path = f'{actmed_img_path}/{image}'
    dst_path = f'{images_dir}/{image}'
    
    image_type = image.split('.')[-1]
    new_image_name = f'x-ray-{xray_index}.{image_type}'
    
    shutil.copy(src_path, dst_path)
    os.rename(dst_path, f'./{images_dir}/{new_image_name}')
    
    finding = row['finding']
    
    if finding == 'COVID-19':
        sample = new_image_name + ',' + 'positive' + '\n'
        covid_positives += 1
    else:
        sample = new_image_name + ',' + 'negative' + '\n'
        covid_negatives += 1
    
    data_file.write(sample)
    xray_index += 1
    
    print(new_image_name, ': OK')

print('Covid positives:', covid_positives)
print('Covid negatives:', covid_negatives)

data_file.close()

(238, 13)
x-ray-298.png : OK
x-ray-299.png : OK
x-ray-300.png : OK
x-ray-301.png : OK
x-ray-302.png : OK
x-ray-303.png : OK
x-ray-304.png : OK
x-ray-305.png : OK
x-ray-306.png : OK
x-ray-307.png : OK
x-ray-308.png : OK
x-ray-309.png : OK
x-ray-310.png : OK
x-ray-311.png : OK
x-ray-312.png : OK
x-ray-313.png : OK
x-ray-314.png : OK
x-ray-315.png : OK
x-ray-316.png : OK
x-ray-317.png : OK
x-ray-318.png : OK
x-ray-319.png : OK
x-ray-320.png : OK
x-ray-321.png : OK
x-ray-322.png : OK
x-ray-323.png : OK
x-ray-324.png : OK
x-ray-325.png : OK
x-ray-326.png : OK
x-ray-327.png : OK
x-ray-328.png : OK
x-ray-329.png : OK
x-ray-330.png : OK
x-ray-331.png : OK
x-ray-332.png : OK
x-ray-333.png : OK
x-ray-334.png : OK
x-ray-335.png : OK
x-ray-336.png : OK
x-ray-337.png : OK
x-ray-338.png : OK
x-ray-339.png : OK
x-ray-340.png : OK
x-ray-341.png : OK
x-ray-342.png : OK
x-ray-343.png : OK
x-ray-344.png : OK
x-ray-345.png : OK
x-ray-346.png : OK
x-ray-347.png : OK
x-ray-348.png : OK
x-ray-349.png : OK
x-

#### Due to potential duplicates and other uncertainties, the datasets that the below cells are exploiting aren't being used as of yet.

### Extract data from 'COVID-19-Radiography-Database'

In [ ]:
sirm_cov_csv = pd.read_excel(sirm_cov_csv_path)
print(sirm_cov_csv.shape)

cohen_urls = set(cohen_csv['url'])

data_file = open(f'./{dataset_dir}/{data_file_name}', 'a')

for _, row in sirm_cov_csv.iterrows():
    if row['URL'] in cohen_urls:
        continue
    
    image_type = row['FORMAT'].lower()
    image = f"COVID ({row['FILE NAME'].split(' ')[1]}).{image_type}"
    
    src_path = f'./{sirm_cov_img_path}/{image}'
    dst_path = f'./{images_dir}/{image}'
    
    new_image_name = f'x-ray-{xray_index}.{image_type}'
    
    shutil.copy2(src_path, dst_path)
    os.rename(dst_path, f'./{images_dir}/{new_image_name}')
    
    sample = new_image_name + ',' + 'positive' + '\n'
    covid_positives += 1
    
    data_file.write(sample)
    xray_index += 1
    
    print(new_image_name, ': OK')

print('Covid positives:', covid_positives)
print('Covid negatives:', covid_negatives)

data_file.close()

In [ ]:
rsna_csv2 = pd.read_csv(rsna_csv2_path, nrows=None)
print(rsna_csv2.shape)

data_file = open(f'./{dataset_dir}/{data_file_name}', 'a')

for _, row in rsna_csv2.iterrows():
    patient_id = row['patientId']
    dcm_path = f'{rsna_img_path}/{patient_id}.dcm'
    
    ds = dicom.dcmread(dcm_path)
    pixel_array = ds.pixel_array
    
    new_image_name = f'x-ray-{xray_index}.png'
    dst_path = f'{images_dir}/{new_image_name}'
    
    cv2.imwrite(dst_path, pixel_array)
    
    sample = new_image_name + ',' + 'negative' + '\n'
    covid_negatives += 1
    
    data_file.write(sample)    
    xray_index += 1
    
    print(new_image_name, ': OK')

print('Covid positives:', covid_positives)
print('Covid negatives:', covid_negatives)

data_file.close()

### Store data in Azure blob storage

In [9]:
from dotenv import load_dotenv
load_dotenv()

load_dotenv(verbose=True)

from pathlib import Path
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

import io

container_name = os.getenv('CONTAINERNAME')
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

try:
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    container_client = blob_service_client.get_container_client(container_name)
    csv_blob_client = blob_service_client.get_blob_client(container=container_name, blob="Covid-19-1/data_file.csv")

    with open("dataset/data_file.csv", "rb") as data:
        csv_blob_client.upload_blob(data, overwrite=True)

    blob_deletions = 0

    img_blob_list = container_client.list_blobs(name_starts_with='Covid-19-1/images/')
    for blob in img_blob_list:
        img_blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob['name'])
        img_blob_client.delete_blob()
        blob_deletions += 1
    print(f'Successfully deleted {blob_deletions} image blobs')
    
    files = glob(f'{images_dir}/*')
    for f in files:
        img_name = f.split('/')[-1]
        img_blob_client = blob_service_client.get_blob_client(container=container_name, blob=f"Covid-19-1/images/{img_name}")

        image_content_setting = ContentSettings(content_type='image/jpeg')
        with open(f, "rb") as data:
            img_blob_client.upload_blob(data, overwrite=True, content_settings=image_content_setting)
        
        print(f'{img_name} : OK')

except Exception as ex:
    print('Exception:', ex)

Successfully deleted 195 image blobs
x-ray-0.jpeg : OK
x-ray-1.jpeg : OK
x-ray-10.jpeg : OK
x-ray-100.jpg : OK
x-ray-101.jpg : OK
x-ray-102.jpg : OK
x-ray-103.jpg : OK
x-ray-104.jpg : OK
x-ray-105.JPG : OK
x-ray-106.jpg : OK
x-ray-107.jpg : OK
x-ray-108.jpg : OK
x-ray-109.jpg : OK
x-ray-11.jpeg : OK
x-ray-110.jpg : OK
x-ray-111.jpg : OK
x-ray-112.jpeg : OK
x-ray-112.jpg : OK
x-ray-113.jpg : OK
x-ray-114.jpg : OK
x-ray-115.jpg : OK
x-ray-116.png : OK
x-ray-117.jpg : OK
x-ray-118.PNG : OK
x-ray-119.png : OK
x-ray-12.jpg : OK
x-ray-120.png : OK
x-ray-121.png : OK
x-ray-122.jpg : OK
x-ray-123.jpg : OK
x-ray-124.jpg : OK
x-ray-125.jpg : OK
x-ray-126.jpg : OK
x-ray-127.jpg : OK
x-ray-128.jpg : OK
x-ray-129.png : OK
x-ray-13.jpeg : OK
x-ray-130.png : OK
x-ray-131.png : OK
x-ray-132.jpg : OK
x-ray-133.jpg : OK
x-ray-134.jpg : OK
x-ray-135.jpg : OK
x-ray-136.jpg : OK
x-ray-137.jpg : OK
x-ray-138.jpg : OK
x-ray-139.jpg : OK
x-ray-14.jpeg : OK
x-ray-140.jpg : OK
x-ray-141.jpg : OK
x-ray-142.jpg :